### roboflow의 tfrecord 파일을 갖고 training 시키는 코드

### 조건!
1. tensorflow 2 이상의 버전
2. module 에러나면 pip install 해주면 됨! 안될경우 pip install 모듈이름 --user
3. 위의 명령어에도 install 되지 않으면.. 몰라 ㅠㅠㅎ
4. !tar 같은 명령어 안되면 걍 수동으로 해주면 됨

In [1]:
import itertools
import os
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
#import tensorflow_hub as hub
print("TF version:", tf.__version__)
#print("Hub version:", hub.__version__) 노 필요
print("GPU is",tf.test.is_gpu_available())
print("GPU name : ",tf.test.gpu_device_name())
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

TF version: 2.3.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is True
GPU name :  /device:GPU:0
GPU is available


In [25]:
# 이미 된 부분
#%cd C:\Users\IVPL-D14\models\research
#!protoc C:\Users\IVPL-D14\models\research\object_detection\protos\*.proto --python_out=.

C:\Users\IVPL-D14\models\research


C:\Users\IVPL-D14\models\research\object_detection\protos\anchor_generator.proto: File does not reside within any path specified using --proto_path (or -I).  You must specify a --proto_path which encompasses this file.  Note that the proto_path must be an exact prefix of the .proto file names -- protoc is too dumb to figure out when two paths (e.g. absolute and relative) are equivalent (it's harder than you think).


In [41]:
# 죽어도 안됨.. ㅋ 그냥 setup.py 직접 copy하도록!
#%cd C:/Users/IVPL-D14/models/research
#%xcopy C:/Users/IVPL-D14/models/research/object_detection/packages/tf2/setup.py C:/Users/IVPL-D14/models/research
    
#!python -m pip install .

C:\Users\IVPL-D14\models\research


UsageError: Line magic function `%xcopy` not found.


In [2]:
import matplotlib
import matplotlib.pyplot as plt
import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
#from object_detection.builders import model_builder  
# 이건 test해보려는 건데 tensor ver1에서만 된다. 굳이 필요없으니 생략하자
%matplotlib inline

### Run model builder test

In [21]:
# test 부분! 노필요! 생략
!python C:/Users/IVPL-D14/models/research/object_detection/builders/model_builder.py

2021-01-13 15:21:46.721767: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cudart64_101.dll


### implement basic functions

In [3]:
def load_image_into_numpy_array(path):
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
    image_np_with_annotations = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_annotations,
        boxes,
        classes,
        scores,
        category_index,
        use_normalized_coordinates=True,
        min_score_thresh=0.8)
    if image_name:
        plt.imsave(image_name, image_np_with_annotations)
    else:
        plt.imshow(image_np_with_annotations)

## 만든 roboflow 경로를 삽입

In [4]:
## 경로는 임의로 정하기
%cd C:/Users/IVPL-D14/roboflow
!curl -L "https://app.roboflow.com/ds/6HCmQO42NX?key=5Ac9DacoNu" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

C:\Users\IVPL-D14\roboflow


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   891  100   891    0     0    891      0  0:00:01  0:00:01 --:--:--   678
100   891  100   891    0     0    891      0  0:00:01  0:00:01 --:--:--   678

  0 51.8M    0   456    0     0    228      0 66:13:18  0:00:02 66:13:16   228
 15 51.8M   15 8208k    0     0  2736k      0  0:00:19  0:00:03  0:00:16 10.9M
 37 51.8M   37 19.2M    0     0  4936k      0  0:00:10  0:00:04  0:00:06 11.1M
 58 51.8M   58 30.4M    0     0  6236k      0  0:00:08  0:00:05  0:00:03 11.1M
 80 51.8M   80 41.7M    0     0  7122k      0  0:00:07  0:00:06  0:00:01 11.1M
 98 51.8M   98 51.2M    0     0  7494k      0  0:00:07  0:00:07 --:--:-- 10.8M
100 51.8M  100 51.8M    0     0  7582k      0  0:0

In [6]:
%pwd
%cd C:/Users/IVPL-D14/roboflow
%ls -al train

C:\Users\IVPL-D14\roboflow
 C 드라이브의 볼륨에는 이름이 없습니다.

파일을 찾을 수 없습니다.



 볼륨 일련 번호: 7EBF-306A

 C:\Users\IVPL-D14\roboflow 디렉터리


 C:\Users\IVPL-D14\roboflow\train 디렉터리

2021-01-30  오후 08:05    <DIR>          .
2021-01-30  오후 08:05    <DIR>          ..
2021-01-30  오후 08:03        46,499,714 kb.tfrecord
2021-01-30  오후 08:03               126 kb_label_map.pbtxt
               2개 파일          46,499,840 바이트
               2개 디렉터리  97,184,763,904 바이트 남음


In [29]:
#test_record_fname = 'C:/Users/IVPL-D14/roboflow/valid/kb.tfrecord'
#train_record_fname = 'C:/Users/IVPL-D14/roboflow/train/kb.tfrecord'
#label_map_pbtxt_fname = 'C:/Users/IVPL-D14/roboflow/train/kb_label_map.pbtxt'

In [3]:
test_record_fname = 'C:/Users/IVPL-D14/models/research/object_detection/output_record/test.record'
train_record_fname = 'C:/Users/IVPL-D14/models/research/object_detection/output_record/train.record'
label_map_pbtxt_fname = 'C:/Users/IVPL-D14/models/research/object_detection/training/labelmap.pbtxt'

In [4]:
## 용량 너무 크다는 에러때문에 batch_size를 8로 대폭 축소함
## batch_size는 최소 8부터 2의 거듭제곱으로 설정한다!
MODELS_CONFIG = {
'efficientdet-d0': {
'model_name': 'efficientdet_d0_coco17_tpu-32',
'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
'batch_size': 8
},
'efficientdet-d1': {
'model_name': 'efficientdet_d1_coco17_tpu-32',
'base_pipeline_file': 'ssd_efficientdet_d1_640x640_coco17_tpu-8.config',
'pretrained_checkpoint': 'efficientdet_d1_coco17_tpu-32.tar.gz',
'batch_size': 8
},
'efficientdet-d2': {
'model_name': 'efficientdet_d2_coco17_tpu-32',
'base_pipeline_file': 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config',
'pretrained_checkpoint': 'efficientdet_d2_coco17_tpu-32.tar.gz',
'batch_size': 16
},
'efficientdet-d3': {
'model_name': 'efficientdet_d3_coco17_tpu-32',
'base_pipeline_file': 'ssd_efficientdet_d3_896x896_coco17_tpu-32.config',
'pretrained_checkpoint': 'efficientdet_d3_coco17_tpu-32.tar.gz',
'batch_size': 16
}
}
#in this tutorial we implement the lightweight, smallest state of the art efficientdet model
#if you want to scale up tot larger efficientdet models you will likely need more compute!
chosen_model = 'efficientdet-d0'
num_steps = 80000 
num_eval_steps = 500   #Perform evaluation after so many steps
model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] #if you can fit a large batch in memory, it may speed up your tr

In [5]:
#num_steps = 10000 
#The more steps, the longer the training. 
#Increase if your loss function is still decreasing and validation metrics are increasing.

#num_eval_steps = 500 
#Perform evaluation after so many steps

### download pre-trained wieghts from web

In [8]:
%mkdir C:\Users\IVPL-D14\models\research\deploy
%cd C:/Users/IVPL-D14/models/research/deploy

C:\Users\IVPL-D14\models\research\deploy


하위 디렉터리 또는 파일 C:\Users\IVPL-D14\models\research\deploy이(가) 이미 있습니다.


In [5]:
%pwd

'C:\\Users\\IVPL-D14\\models\\research\\object_detection'

In [9]:
import wget
import tarfile

In [13]:
#다양한 모델 다운받는 곳! 이건 따로 다운받을때만 실행하기

download_tar='http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_coco_2018_01_28.tar.gz'
wget.download(download_tar)
tar = tarfile.open('faster_rcnn_resnet50_coco_2018_01_28.tar.gz')
tar.extractall()
tar.close()

NameError: name 'wget' is not defined

In [14]:
import wget
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
wget.download(download_tar)
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

  6% [....                                                                    ]  2007040 / 30736482

100% [........................................................................] 30736482 / 30736482

In [15]:
%cd C:/Users/IVPL-D14/models/research/deploy
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
wget.download(download_config)

C:\Users\IVPL-D14\models\research\deploy
100% [................................................................................] 4630 / 4630

'ssd_efficientdet_d0_512x512_coco17_tpu-8.config'

### prepare some info for training

In [5]:
pipeline_fname = 'C:/Users/IVPL-D14/models/research/deploy/' + base_pipeline_file
fine_tune_checkpoint = 'C:/Users/IVPL-D14/models/research/deploy/' + model_name + '/checkpoint/ckpt-0'

In [6]:
print(pipeline_fname)
print(fine_tune_checkpoint)

C:/Users/IVPL-D14/models/research/deploy/ssd_efficientdet_d0_512x512_coco17_tpu-8.config
C:/Users/IVPL-D14/models/research/deploy/efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0


In [7]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=2, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

num_classes = get_num_classes(label_map_pbtxt_fname)

In [8]:
import re

%cd C:\Users\IVPL-D14\models\research\deploy
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    s = re.sub('fine_tune_checkpoint: ".*?"','fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
# tfrecord files train and test.
    s = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)
# label_map_path
    s = re.sub('label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname),s)
# Set training batch_size.
    s = re.sub('batch_size: [0-9]+','batch_size: {}'.format(batch_size), s)
# Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+','num_steps: {}'.format(num_steps), s)
# Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+','num_classes: {}'.format(num_classes), s)
#fine-tune checkpoint type
    s = re.sub('fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)

    f.write(s)

C:\Users\IVPL-D14\models\research\deploy
writing custom configuration file


In [9]:
%pycat C:\Users\IVPL-D14\models\research\deploy\pipeline_file.config

In [10]:
pipeline_file = 'C:/Users/IVPL-D14/models/research/deploy/pipeline_file.config'
model_dir = 'C:/Users/IVPL-D14/training_kb_0215_re80000_more_bats/'

### 만약 tensorflow 이상한 에러가 뜬다면 아래 import 수행하기

In [18]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [14]:
%ls -al

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 7EBF-306A

 C:\Users\IVPL-D14\models\research\deploy 디렉터리



파일을 찾을 수 없습니다.


In [19]:
import tensorflow as tf
print(tf.__version__)

2.3.0


## 학습!
#### 학습될때 딱히 colab 처럼 아래에 진행상황이 뜨지않고 [*] 인 상태이므로 혹시 모르면 
#### 로컬폴더가서  ckpt 파일이 생성이 되고있는지 확인하면 된다

In [11]:
!python C:/Users/IVPL-D14/models/research/object_detection/model_main_tf2.py \
        --pipeline_config_path={pipeline_file} \
        --model_dir={model_dir} \
        --alsologtostderr \
        --num_train_steps={num_steps} \
        --sample_1_of_n_eval_examples=1 \
        --num_eval_steps={num_eval_steps}

2021-02-15 14:49:08.325291: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cudart64_101.dll
2021-02-15 14:49:19.210067: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library nvcuda.dll
2021-02-15 14:49:19.241525: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1070 computeCapability: 6.1
coreClock: 1.7465GHz coreCount: 15 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 238.66GiB/s
2021-02-15 14:49:19.241557: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cudart64_101.dll
2021-02-15 14:49:19.247829: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cublas64_10.dll
2021-02-15 14:49:19.251500: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cufft64_10.dll
2

Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
2021-02-15 14:50:08.870880: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cublas64_10.dll
2021-02-15 14:50:09.091602: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cudnn64_7.dll
2021-02-15 14:50:09.818359: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Invoking GPU asm compilation is supported on Cuda non-Windows platforms only
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
W0215 14:50:15.999474  6044 util.py:150] Unresolved object in checkpoint: (root).model._groundtruth_lists
W0215 14:50:16.000474  6044 util.py:150] Unresolved object in checkpoint: (root).model._box_predictor
W0215 14:50:16.000474  6044 util.py:150] Unresolved object in checkpoint: (root).model._batched_p

W0215 14:50:16.013469  6044 util.py:150] Unresolved object in checkpoint: (root).model._box_predictor._base_tower_layers_for_heads.box_encodings.4.7.axis
W0215 14:50:16.013469  6044 util.py:150] Unresolved object in checkpoint: (root).model._box_predictor._base_tower_layers_for_heads.box_encodings.4.7.gamma
W0215 14:50:16.013469  6044 util.py:150] Unresolved object in checkpoint: (root).model._box_predictor._base_tower_layers_for_heads.box_encodings.4.7.beta
W0215 14:50:16.013469  6044 util.py:150] Unresolved object in checkpoint: (root).model._box_predictor._base_tower_layers_for_heads.box_encodings.4.7.moving_mean
W0215 14:50:16.014469  6044 util.py:150] Unresolved object in checkpoint: (root).model._box_predictor._base_tower_layers_for_heads.box_encodings.4.7.moving_variance
W0215 14:50:16.014469  6044 util.py:150] Unresolved object in checkpoint: (root).model._box_predictor._base_tower_layers_for_heads.class_predictions_with_background.0.1.axis
W0215 14:50:16.014469  6044 util.py:1

I0216 03:02:44.915548  6044 model_lib_v2.py:651] Step 63900 per-step time 0.641s loss=0.462
INFO:tensorflow:Step 64000 per-step time 0.693s loss=0.394
I0216 03:03:53.836715  6044 model_lib_v2.py:651] Step 64000 per-step time 0.693s loss=0.394
INFO:tensorflow:Step 64100 per-step time 0.700s loss=0.175
I0216 03:05:02.778183  6044 model_lib_v2.py:651] Step 64100 per-step time 0.700s loss=0.175
INFO:tensorflow:Step 64200 per-step time 0.656s loss=0.197
I0216 03:06:10.446478  6044 model_lib_v2.py:651] Step 64200 per-step time 0.656s loss=0.197
INFO:tensorflow:Step 64300 per-step time 0.619s loss=0.192
I0216 03:07:17.435229  6044 model_lib_v2.py:651] Step 64300 per-step time 0.619s loss=0.192
INFO:tensorflow:Step 64400 per-step time 0.669s loss=0.215
I0216 03:08:24.544337  6044 model_lib_v2.py:651] Step 64400 per-step time 0.669s loss=0.215
INFO:tensorflow:Step 64500 per-step time 0.694s loss=0.172
I0216 03:09:31.763877  6044 model_lib_v2.py:651] Step 64500 per-step time 0.694s loss=0.172
IN

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 10252.

In [12]:
%ls  C:\Users\IVPL-D14\training_kb_0215_re80000_more_bats

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 7EBF-306A

 C:\Users\IVPL-D14\training_kb_0215_re80000_more_bats 디렉터리

2021-02-16  오전 06:03    <DIR>          .
2021-02-16  오전 06:03    <DIR>          ..
2021-02-16  오전 06:03               705 checkpoint
2021-02-16  오전 04:56        44,565,486 ckpt-75.data-00000-of-00001
2021-02-16  오전 04:56            89,701 ckpt-75.index
2021-02-16  오전 05:07        44,565,486 ckpt-76.data-00000-of-00001
2021-02-16  오전 05:07            89,701 ckpt-76.index
2021-02-16  오전 05:18        44,565,486 ckpt-77.data-00000-of-00001
2021-02-16  오전 05:18            89,701 ckpt-77.index
2021-02-16  오전 05:29        44,565,486 ckpt-78.data-00000-of-00001
2021-02-16  오전 05:29            89,701 ckpt-78.index
2021-02-16  오전 05:40        44,565,486 ckpt-79.data-00000-of-00001
2021-02-16  오전 05:40            89,701 ckpt-79.index
2021-02-16  오전 05:51        44,565,486 ckpt-80.data-00000-of-00001
2021-02-16  오전 05:51            89,701 ckpt-80.index
2021-02-16  오전 06:03        44,565,486 ck

In [13]:
import re
import numpy as np
output_directory = 'C:/Users/IVPL-D14/FineTunedModels/training_kb_0215_re80000_more_bats_fintuned_model'
#place the model weights you would like to export here
last_model_path = 'C:/Users/IVPL-D14/training_kb_0215_re80000_more_bats'
print(last_model_path)
!python C:/Users/IVPL-D14/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}

C:/Users/IVPL-D14/training_kb_0215_re80000_more_bats


2021-02-16 11:57:52.790196: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cudart64_101.dll
2021-02-16 11:58:46.094735: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library nvcuda.dll
2021-02-16 11:58:46.321141: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1070 computeCapability: 6.1
coreClock: 1.7465GHz coreCount: 15 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 238.66GiB/s
2021-02-16 11:58:46.321171: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cudart64_101.dll
2021-02-16 11:58:48.286271: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cublas64_10.dll
2021-02-16 11:58:51.420067: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cufft64_10.dll
2

In [19]:
#check check
%ls -al C:\Users\IVPL-D14\FineTunedModels\368_batch8_finetuned_model\saved_model

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 7EBF-306A

 C:\Users\IVPL-D14\models\research\deploy 디렉터리


 C:\Users\IVPL-D14\fine_tuned_models\511_batch8_finetuned_model\saved_model 디렉터리

2021-01-16  오후 09:59    <DIR>          .

파일을 찾을 수 없습니다.



2021-01-16  오후 09:59    <DIR>          ..
2021-01-16  오후 09:59    <DIR>          assets
2021-01-16  오후 09:59        19,137,241 saved_model.pb
2021-01-16  오후 09:59    <DIR>          variables
               1개 파일          19,137,241 바이트
               4개 디렉터리  349,916,651,520 바이트 남음


In [21]:
%ls -al C:\Users\IVPL-D14\FineTunedModels\511_batch8_finetuned_model\saved_model\variables

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 7EBF-306A

 C:\Users\IVPL-D14\models\research\deploy 디렉터리


 C:\Users\IVPL-D14\FineTunedModels\511_batch8_finetuned_model\saved_model\variables 디렉터리

2021-01-16  오후 09:59    <DIR>          .
2021-01-16  오후 09:59    <DIR>          ..
2021-01-16  오후 09:59        22,614,828 variables.data-00000-of-00001
2021-01-16  오후 09:59            39,718 variables.index
               2개 파일          22,654,546 바이트
               2개 디렉터리  349,917,728,768 바이트 남음


파일을 찾을 수 없습니다.


In [60]:
import os
output_directory = 'C:/Users/IVPL-D14/fine_tuned_model/saved_model/'
pb_fname = os.path.join(os.path.abspath(output_directory), "saved_model.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)

### download 하는 부분!
google.colab 모듈을 불러올수 없는 문제도 있지만.. 이미 로컬에 저장되어있으므로 굳이
다운받지 않고 사용하면 됨..!

In [86]:
#label_map_pbtxt_fname = 'C:/Users/IVPL-D14/models/train/knife_label_map.pbtxt'
#from google.colab import files
#files.download(label_map_pbtxt_fname)

### 이 명령어도 듣지 않는다.
대신 앞서 만든 deploy 폴더안의 tar 파일을 fine_tuned_model에 가져와서 압축해제 해주면 된다!


#%pwd

#!tar -cvzf trained_model_large_original_20000.tar.gz C:/Users/IVPL-D14/fine_tuned_model/

In [90]:
## test 해주는 부분 돌리면 됨.. !